In [1]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import numpy as np
import tensorflow as tf
import h5py

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
from tensorflow.keras.layers import Layer, Lambda, Permute

In [3]:
model_weights1 = 'model_val_accuracy.h5'
dataset_path = 'test_00001.h5'
batch_size = 64

In [4]:
class BilinearInterpolate3D(Layer):
    """
    Bi-linearly interpolates over the 
    """
    def __init__(self, output_shape, **kwargs):
        """
        output_shape: shape of the output tensor(this is considered to be channel_last)
        """
        assert(len(output_shape) == 3)
        super(BilinearInterpolate3D, self).__init__(**kwargs)

        self._output_shape = np.array(output_shape)

        # setting up the lambda layer used for bilinear interpolation
        def resize_bilinear(tensor, output_shape): return tf.image.resize_bilinear(tensor, output_shape, align_corners=True)
        self._bilinear_interpolation_x_y = Lambda(resize_bilinear, arguments={'output_shape':self._output_shape[0:2]})
        self._permute_x_z = Permute((3,2,1))
        self._bilinear_interpolation_z_y = Lambda(resize_bilinear, arguments={'output_shape':self._output_shape[1:3][::-1]})
        self._permute_z_x = Permute((3,2,1))

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        assert(len(input_shape) == 4)
        super(BilinearInterpolate3D, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        # logic executed when the layer is called
        assert(len(x.shape) == 4)
        x = self._bilinear_interpolation_x_y(x)
        x = self._permute_x_z(x)
        x = self._bilinear_interpolation_z_y(x)
        x = self._permute_z_x(x)
        return x 

    def get_config(self):
        # need to override this
        config = super(BilinearInterpolate3D, self).get_config().copy()
        config.update({
           'output_shape': self._output_shape
        })
        return config

In [5]:
from tensorflow.keras.models import load_model

In [6]:
model1 = load_model(model_weights1, custom_objects={'BilinearInterpolate3D':BilinearInterpolate3D})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
# Compiling the model
opt_rms = RMSprop(lr=0.001,decay=1e-6)
model1.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

In [8]:
def read_h5_dataset(filename):
    'Reads a h5 dataset. and returns it'
    with h5py.File(filename, 'r') as h5File:
        if 'dataset' not in h5File.keys(): raise ValueError('h5 file has no dataset')
        dataset = h5File['dataset']
        x_set = (np.array(dataset['x_set'])*255-120.84449672851562)/64.13596441053241
        y_set = np.array(dataset['y_set'])
    return x_set, y_set

In [9]:
# Load dataset
x_test, y_test = read_h5_dataset(dataset_path)

In [10]:
model1.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 80)        2240      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32, 32, 80)        320       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 80)        0         
_________________________________________________________________
bilinear_interpolate3d (Bili (None, 75, 75, 75)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 75)        50700     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 75, 75, 75)        300 

In [11]:
# Evaluating 
scores = model1.evaluate(x_test, y_test, batch_size=batch_size)

10000/10000 [==============================] - 9s 899us/sample - loss: 0.4911 - acc: 0.8815
